In [30]:
import pandas as pd

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from scipy.stats import rankdata, norm

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

import warnings 
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("data.csv")

In [3]:
y = df["Income"]

In [4]:
df["Work_Experience_rank"] = rankdata(df["Work_Experience"])
df["Work_Experience_rank_norm"] = norm.ppf((df["Work_Experience_rank"] - 0.5) / len(df["Work_Experience_rank"]))

In [5]:
X = df[["Work_Experience_rank_norm", 'Education_Level', 'Occupation', 'Location', 'Employment_Status',
        'Homeownership_Status', 'Type_of_Housing']]

In [6]:
X

,Work_Experience_rank_norm,Education_Level,Occupation,Location,Employment_Status,Homeownership_Status,Type_of_Housing
0,-0.194863,Master's,Technology,Urban,Full-time,Own,Apartment
1,-1.331575,High School,Finance,Urban,Full-time,Own,Apartment
2,-1.886705,Bachelor's,Technology,Urban,Full-time,Own,Single-family home
3,0.356989,High School,Others,Urban,Full-time,Own,Apartment
4,-0.515076,Bachelor's,Finance,Urban,Self-employed,Own,Townhouse
...,...,...,...,...,...,...,...
9995,0.056052,High School,Others,Suburban,Full-time,Own,Townhouse
9996,-0.459605,Master's,Technology,Urban,Full-time,Own,Single-family home
9997,2.393247,Doctorate,Education,Urban,Full-time,Own,Single-family home
9998,-1.461421,High School,Education,Urban,Part-time,Own,Single-family home


In [7]:
X = pd.get_dummies(X, columns=["Occupation", "Location", "Employment_Status", "Homeownership_Status", "Type_of_Housing"],
                  dtype="int8")
X["Education_Level"] = X["Education_Level"].replace({"High School":0,
                                                        "Bachelor's":1,
                                                        "Master's":2,
                                                        "Doctorate":3})

/tmp/ipykernel_2604/391221571.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X["Education_Level"] = X["Education_Level"].replace({"High School":0,


In [8]:
test_data_x = X.iloc[9951:]
test_data_y = y.iloc[9951:]

In [9]:
X = X[:9950]
y = y[:9950]

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75)

ValueError: Expected a 2-dimensional container but got <class 'pandas.core.series.Series'> instead. Pass a DataFrame containing a single row (i.e. single sample) or a single column (i.e. single feature) instead.

In [32]:
import statsmodels.api as sm

X_train_constant = sm.add_constant(X_train)
lr = sm.OLS(y_train, X_train_constant).fit()

In [33]:
lr.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Income   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     6.740
Date:                Wed, 10 Jul 2024   Prob (F-statistic):           4.84e-13
Time:                        17:53:40   Log-Likelihood:            -1.1813e+05
No. Observations:                7462   AIC:                         2.363e+05
Df Residuals:                    7448   BIC:                         2.364e+05
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
======================================================================================================
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
const                               3.673e+05    1.5e+04     24.515      0.000    3.38e+05    3.97e+05
Work_Experience_rank_norm          -4.549e+04   2.13e+04     -2.132      0.033   -8.73e+04   -3661.540
Education_Level                    -5.393e+04   2.45e+04     -2.197      0.028   -1.02e+05   -5802.946
Occupation_Education                1.592e+05   4.74e+04      3.356      0.001    6.62e+04    2.52e+05
Occupation_Finance                 -5.862e+04   4.73e+04     -1.239      0.215   -1.51e+05    3.41e+04
Occupation_Healthcare               5.253e+04   3.66e+04      1.435      0.151   -1.92e+04    1.24e+05
Occupation_Others                   1.243e+05   4.73e+04      2.625      0.009    3.15e+04    2.17e+05
Occupation_Technology               8.997e+04   3.99e+04      2.255      0.024    1.18e+04    1.68e+05
Location_Rural                      2.447e+05   4.95e+04      4.947      0.000    1.48e+05    3.42e+05
Location_Suburban                   1.523e+05   3.98e+04      3.830      0.000    7.43e+04     2.3e+05
Location_Urban                     -2.968e+04   2.99e+04     -0.992      0.321   -8.84e+04     2.9e+04
Employment_Status_Full-time         8.085e+04   2.79e+04      2.894      0.004    2.61e+04    1.36e+05
Employment_Status_Part-time         2.022e+05   3.19e+04      6.336      0.000     1.4e+05    2.65e+05
Employment_Status_Self-employed      8.43e+04   3.66e+04      2.306      0.021    1.26e+04    1.56e+05
Homeownership_Status_Own            5.648e+04   2.21e+04      2.557      0.011    1.32e+04    9.98e+04
Homeownership_Status_Rent           3.109e+05   2.35e+04     13.238      0.000    2.65e+05    3.57e+05
Type_of_Housing_Apartment           1.735e+05   2.94e+04      5.894      0.000    1.16e+05    2.31e+05
Type_of_Housing_Single-family home   1.11e+05   2.91e+04      3.814      0.000     5.4e+04    1.68e+05
Type_of_Housing_Townhouse           8.284e+04   3.67e+04      2.259      0.024     1.1e+04    1.55e+05
==============================================================================
Omnibus:                     4491.817   Durbin-Watson:                   1.996
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            32932.792
Skew:                           2.943   Prob(JB):                         0.00
Kurtosis:                      11.442   Cond. No.                     3.24e+16
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 3.1e-29. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [11]:
dtr = DecisionTreeRegressor()
dtr.fit(X_train, y_train)

DecisionTreeRegressor()

In [22]:
rf = RandomForestRegressor()
rf.fit(X_train, y_train)

RandomForestRegressor()

In [23]:
prediction_rf = rf.predict(X_test)

In [12]:
prediction = dtr.predict(X_test)

In [27]:
scores = cross_val_score(dtr, X_train, y_train, cv=5, scoring='r2')
result = -scores.mean()
print(result)

0.7101431710603631


In [25]:
mean_absolute_error(y_test, prediction_rf)

1034726.3044384563

In [26]:
r2_score(y_test, prediction_rf)

-0.026057986068011907

In [17]:
datapoint = [list(test_data_x.iloc[3].values)]

In [18]:
dtr.predict(datapoint)

/home/murad/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(


array([124822.])

In [19]:
test_data_y.iloc[3]

2735664

In [28]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import numpy as np

# Assuming X_train and y_train are defined
# List of models to try
models = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(),
    "Lasso Regression": Lasso(),
    "ElasticNet Regression": ElasticNet(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest": RandomForestRegressor(),
    "Gradient Boosting": GradientBoostingRegressor(),
    "XGBoost": XGBRegressor(),
    "LightGBM": LGBMRegressor(),
    "CatBoost": CatBoostRegressor(verbose=0),
    "SVR": SVR(),
    "KNN": KNeighborsRegressor()
}

# Train and evaluate each model using R2 score
r2_results = {}
for name, model in models.items():
    r2_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='r2')
    r2_results[name] = r2_scores.mean()

# Print results
for name, score in r2_results.items():
    print(f"{name}: R2 Score = {score}")


/home/murad/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.512e+14, tolerance: 2.019e+12
  model = cd_fast.enet_coordinate_descent(
/home/murad/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.476e+14, tolerance: 2.056e+12
  model = cd_fast.enet_coordinate_descent(
/home/murad/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality 

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000270 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 88
[LightGBM] [Info] Number of data points in the train set: 5969, number of used features: 18
[LightGBM] [Info] Start training from score 827275.007371
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000250 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 88
[LightGBM] [Info] Number of data points in the train set: 5969, number of used features: 18
[LightGBM] [Info] Start training from score 840523.907020
[LightGBM] [Warning] Fou